In [ ]:
#| default_exp explain

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2
from ipynb_path import *

In [ ]:
#| export
from __future__ import annotations
from relax.import_essentials import *
from relax.data_module import DataModule, load_data
from relax.base import *
from relax.methods import *
from relax.strategy import *
from relax.ml_model import *
import einops
from sklearn.datasets import make_classification

Using JAX backend.


No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


## Explanation

In [ ]:
#| export
class Explanation:
    """Generated CF Explanations class. It behaves like a `DataModule`, except a few more attributes."""

    def __init__(
        self,
        data: DataModule,  # Data module
        cfs: Array,  # Generated cf explanation of `xs` in `data`
        pred_fn: Callable[[Array], Array],  # Predict function
        total_time: float = None,  # Total runtime
        cf_name: str = "CFModule",  # CF method's name
    ):
        self._data = data
        self._cfs = cfs
        self.pred_fn = pred_fn
        self.total_time = total_time
        self.cf_name = cf_name

    def __repr__(self):
        return f"Explanation(data_name={self.data_name}, cf_name={self.cf_name}, " \
               f"total_time={self.total_time}, xs={self.xs}, ys={self.ys}, cfs={self.cfs})"

    @property
    def data(self):
        return self._data

    @property
    def xs(self):
        return self.data.xs
    
    @property
    def ys(self):
        return self.data.ys
    
    @property
    def cfs(self) -> Array:
        """Return the counterfactuals in the shape of (n, c, k)"""
        if self._cfs.ndim == 2:
            return einops.rearrange(self._cfs, "n d -> n () d")
        return self._cfs
    
    @property
    def data_name(self):
        return self.data.name

    @property
    def train_indices(self):
        return self.data.train_indices
    
    @property
    def test_indices(self):
        return self.data.test_indices
    
    def apply_constraints(self, *args, **kwargs):
        return self.data.apply_constraints(*args, **kwargs)
    
    def compute_reg_loss(self, *args, **kwargs):
        return self.data.compute_reg_loss(*args, **kwargs)

In [ ]:
#| export
def fake_explanation(n_cfs: int=1):
    dm = load_data('dummy')
    ml_model = load_ml_module('dummy')
    if n_cfs < 1: 
        raise ValueError(f'n_cfs must be greater than 0, but got n_cfs={n_cfs}.')
    elif n_cfs == 1:
        cfs = dm.xs
    else:
        # Allow for multiple counterfactuals
        cfs = einops.repeat(dm.xs, "n k -> n c k", c=n_cfs)

    return Explanation(
        data=dm, cfs=cfs, pred_fn=ml_model.pred_fn, total_time=0.0, cf_name='dummy_method'
    )

In [ ]:
exp = fake_explanation(n_cfs=1)
xs_shape = exp.xs.shape
assert exp.cfs.shape == (xs_shape[0], 1, xs_shape[-1])

exp = fake_explanation(n_cfs=5)
assert exp.cfs.shape == (xs_shape[0], 5, xs_shape[-1])

## Generate Explanations

In [ ]:
#| export
def prepare_pred_fn(
    cf_module: CFModule,
    data: DataModule,
    pred_fn: Callable[[Array, ...], Array], # Predictive function. 
    pred_fn_args: Dict = None,
) -> Callable[[Array], Array]: # Return predictive function with signature `(x: Array) -> Array`.
    """Prepare the predictive function for the CF module. 
    We will train the model if `pred_fn` is not provided and `cf_module` does not have `pred_fn`.
    If `pred_fn` is found in `cf_module`, we will use it irrespective of `pred_fn` argument.
    If `pred_fn` is provided, we will use it.
    """
    # Train the model if `pred_fn` is not provided.
    if not hasattr(cf_module, 'pred_fn') and pred_fn is None:
        model = MLModule().train(data)
        return model.pred_fn
    # If `pred_fn` is detected in cf_module, 
    # use it irrespective of `pred_fn` argument.
    elif hasattr(cf_module, 'pred_fn'):
        return cf_module.pred_fn
    # If `pred_fn` is provided, use it.
    else:
        if pred_fn_args is not None:
            pred_fn = ft.partial(pred_fn, **pred_fn_args)
        return pred_fn

def prepare_cf_module(
    cf_module: CFModule,
    data_module: DataModule,
    pred_fn: Callable[[Array], Array] = None,
    train_config: Dict[str, Any] = None, 
):
    """Prepare the CF module. 
    It will hook up the data module, 
    and its apply functions via the `init_apply_fns` method
    (e.g., `apply_constraints_fn` and `compute_reg_loss_fn`).
    Next, it will train the model if `cf_module` is a `ParametricCFModule`.
    Finally, it will call `before_generate_cf` method.
    """
    cf_module.set_data_module(data_module)
    cf_module.set_apply_constraints_fn(data_module.apply_constraints)
    cf_module.set_compute_reg_loss_fn(data_module.compute_reg_loss)
    if isinstance(cf_module, ParametricCFModule):
        cf_module.train(data_module, pred_fn=pred_fn, **train_config)
    cf_module.before_generate_cf()
    return cf_module


In [ ]:
#| export
def generate_cf_explanations(
    cf_module: CFModule, # CF Explanation Module
    data: DataModule, # Data Module
    pred_fn: Callable[[Array, ...], Array] = None, # Predictive function
    strategy: str | BaseStrategy = None, # Parallelism Strategy for generating CFs. Default to `vmap`.
    train_config: Dict[str, Any] = None, 
    pred_fn_args: dict = None # auxiliary arguments for `pred_fn` 
) -> Explanation: # Return counterfactual explanations.
    """Generate CF explanations."""

    # Prepare `pred_fn`, `cf_module`, and `strategy`.
    pred_fn = prepare_pred_fn(cf_module, data, pred_fn, pred_fn_args)
    cf_module = prepare_cf_module(cf_module, data, train_config)
    if strategy is None:
        strategy = StrategyFactory.get_default_strategy()
    strategy = StrategyFactory.get_strategy(strategy)
    
    # Generate CF explanations.
    start_time = time.time()
    cfs = strategy(cf_module.generate_cf, data.xs, pred_fn).block_until_ready()
    total_time = time.time() - start_time

    # Return CF explanations.
    return Explanation(
        cf_name=cf_module.name,
        data=data,
        cfs=cfs,
        total_time=total_time,
        pred_fn=pred_fn,
    )

In [ ]:
dm = load_data("adult")
ml_model = MLModule().train(dm, epochs=3)

Epoch 1/3
191/191 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7335 - loss: 0.5497    
Epoch 2/3
191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 988us/step - accuracy: 0.8045 - loss: 0.4213
Epoch 3/3
191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 989us/step - accuracy: 0.8149 - loss: 0.4002


In [ ]:
exps = generate_cf_explanations(
    VanillaCF(),
    dm, ml_model.pred_fn,
)

  0%|          | 0/100 [00:00<?, ?it/s]